In [23]:
import pandas as pd

In [24]:
import copy
import os

import pandas as pd
from joblib import Parallel,delayed

from dict import set_simple, main_simple, eff_simple
from rarity import rarity_of_build

proDir = 'D:/yuanshen/artifacts-Filter'


raw_build_data = pd.read_csv(proDir + '/config/build.csv')
raw_build_data.dropna(how='all',axis=1)
raw_position_weights = pd.read_csv(proDir + '/config/position_weights.csv')
raw_position_weights.dropna(how='all',axis=1)
score_weights = raw_position_weights.set_index('position')  # 评分权重
positions = ['flower', 'feather', 'sand', 'cup', 'head']

In [25]:
def expand(build: pd.Series):
    temp = build.fillna(value=0)
    # 获得buildName
    if temp[3] == 0:
        set_name = temp[2]
    else:
        set_name = '{}、{}'.format(temp[2], temp[3])
    if temp[1] == 0:
        build_name = '{}--{}--<{}{}{}>'.format(temp[0], set_name, temp[4], temp[5], temp[6])
    else:
        build_name = '{}--{}--{}--<{}{}{}>'.format(temp[0], temp[1], set_name, temp[4], temp[5], temp[6])
    expanded_build = pd.Series(build_name, index=['buildName'])
    # 获得sets
    sets = []
    sets += set_simple[temp[2]]
    if temp[3] != 0:
        sets += set_simple[temp[3]]
    expanded_build['sets'] = sets
    # 获得各部位主属性列表
    flower_main_att = ['flower', 'lifeStatic']
    feather_main_att = ['feather', 'attackStatic']
    sand_main_att = ['sand'] + main_simple[build[4]]
    cup_main_att = ['cup'] + main_simple[build[5]]
    head_main_att = ['head'] + main_simple[build[6]]
    # 生成有效属性列表
    temp = build[7:14].dropna(inplace=False).index.tolist()
    eff_attribute = []
    for simple in temp:
        eff_attribute.append(eff_simple[simple])
    # 生成build毕业难度列表
    expanded_build['flowerDifficulty'] = rarity_of_build(flower_main_att, eff_attribute)
    expanded_build['featherDifficulty'] = rarity_of_build(feather_main_att, eff_attribute)
    expanded_build['sandDifficulty'] = rarity_of_build(sand_main_att, eff_attribute)
    expanded_build['cupDifficulty'] = rarity_of_build(cup_main_att, eff_attribute)
    expanded_build['headDifficulty'] = rarity_of_build(head_main_att, eff_attribute)
    # 生成各部位主属性评分权重 *5
    expanded_build['flowerMainWeights'] = pd.Series([1, 0], index=flower_main_att)
    expanded_build['featherMainWeights'] = pd.Series([1, 0], index=feather_main_att)
    expanded_build['sandMainWeights'] = pd.Series([1, 3.3], index=sand_main_att)
    expanded_build['cupMainWeights'] = pd.Series([1, 3.3], index=cup_main_att)
    if len(head_main_att) == 2:
        expanded_build['headMainWeights'] = pd.Series([1, 3.3], index=head_main_att)
    else:
        expanded_build['headMainWeights'] = pd.Series([1, 3.3, 3.3], index=head_main_att)
    # 各部位主属性评分权重补完 *3
    for position in positions[2:]:
        temp = '{}MainWeights'.format(position)
        inborn = copy.deepcopy(score_weights.loc[position]).dropna()
        for eff in eff_attribute:  # 对于每一个有效属性
            if eff in expanded_build[temp]:  # drop掉存在于已有权重中的对应【固有】主属性
                inborn.drop(eff, inplace=True)
        for att in inborn.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
            if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
                inborn.drop(att, inplace=True)
        expanded_build[temp] = pd.concat([expanded_build[temp], inborn])
    # 生成副属性评分权重
    temp_sec = copy.deepcopy(score_weights.loc['sec']).dropna()
    for att in temp_sec.index.tolist():
        if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
            temp_sec.drop(att, inplace=True)
    expanded_build['secWeights'] = temp_sec
    # 计算最佳得分
    for position in positions:
        temp = '{}MainWeights'.format(position)
        eff_main = expanded_build[temp][1:2]
        sec = copy.deepcopy(score_weights.loc['sec']).dropna()
        for att in sec.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
            if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
                sec.drop(att, inplace=True)
        for att in sec.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
            if att in eff_main:  # 如果它存在于主属性中，则drop掉
                sec.drop(att, inplace=True)
        if len(sec) > 4:
            full_sec = sec.sort_values(ascending=False)[0:4]
        else:
            full_sec = sec.sort_values(ascending=False)
        best_att = pd.concat([eff_main, full_sec])
        best_score = eff_main[0] + 1.176 * full_sec.sum()
        expanded_build['best_{}'.format(position)] = best_score
    # 返回
    return expanded_build

In [31]:
print('正在展开build......')
raw_build_data = raw_build_data[raw_build_data['启用']=='yes']
build_df = Parallel(n_jobs=8,backend='loky')(delayed(expand)(row) for i,row in raw_build_data.iterrows())
build_df = pd.DataFrame(build_df)

print('展开完成.')

正在展开build......


TypeError: to_pickle() got an unexpected keyword argument 'path_or_buf'

In [32]:
build_df.to_pickle(proDir + '/config/expend_build.pkl')

In [33]:
new_build_df = pd.read_pickle(proDir + '/config/expend_build.pkl')


In [34]:
new_build_df

,buildName,sets,flowerDifficulty,featherDifficulty,sandDifficulty,cupDifficulty,headDifficulty,flowerMainWeights,featherMainWeights,sandMainWeights,cupMainWeights,headMainWeights,secWeights,best_flower,best_feather,best_sand,best_cup,best_head
0,迪卢克--狼末--魔女--<精火双暴>,[crimsonWitch],5.221756,5.221756,7.030681,8.545382,6.194278,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 elementalMastery 3....,cup 1.0 fireBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 elementalMastery ...,5.487216,5.487216,7.611216,8.787216,7.219608
1,迪卢克--螭骨--魔女--<攻火暴率>,[crimsonWitch],5.221756,5.221756,6.144429,8.545382,6.820748,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 attackPercentage 3....,cup 1.0 fireBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 elementalMastery ...,5.487216,5.487216,7.611216,8.787216,7.219608
2,胡桃--护摩--魔女、追忆--<生火暴伤>,"[crimsonWitch, shimenawaReminiscence]",6.676374,6.676374,6.633306,10.000000,7.322430,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 lifePercentage 3....,cup 1.0 fireBonus 3....,head 1.0 criticalDamage 3....,attackPercentage 1.000 lifePercentage ...,5.487216,5.487216,8.787216,8.787216,8.395608
3,胡桃--匣里--魔女、追忆--<生火双暴>,"[crimsonWitch, shimenawaReminiscence]",6.676374,6.676374,6.633306,10.000000,6.695960,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 lifePercentage 3....,cup 1.0 fireBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 lifePercentage ...,5.487216,5.487216,8.787216,8.787216,8.395608
4,宵宫--飞雷--追忆--<攻火暴率>,[shimenawaReminiscence],4.798228,4.798228,5.941730,7.961775,6.618910,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 attackPercentage 3....,cup 1.0 fireBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 critical ...,4.311216,4.311216,6.435216,7.611216,6.043608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,诺艾尔--华馆--<防岩双暴>,[huskOfOpulentDreams],5.221756,5.221756,6.144429,8.545382,6.194278,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 defendPercentage 3....,cup 1.0 rockBonus 3....,head 1.0 critical 3....,defendPercentage 1.000 recharge ...,5.487216,5.487216,7.611216,8.787216,7.219608
65,五郎--华馆、绝缘--<充防防>,"[huskOfOpulentDreams, emblemOfSeveredFate]",4.148208,4.148208,6.218439,5.630042,5.505827,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 recharge 3....,cup 1.0 defendPercentage 3....,head 1.0 defendPercentage 3....,defendPercentage 1.0 recharge 1....,2.352000,2.352000,4.476000,4.476000,4.476000
66,提纳里--饰金--<精草双暴>,[GildedDreams],5.221756,5.221756,7.030681,8.545382,6.194278,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 elementalMastery 3....,cup 1.0 dendroBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 elementalMastery ...,5.487216,5.487216,7.611216,8.787216,7.219608
67,科莱--草套--<攻草双暴>,[DeepwoodMemories],6.676374,6.676374,6.633984,10.000000,6.695960,flower 1 lifeStatic 0 dtype: int64,feather 1 attackStatic 0 dtype: int64,sand 1.0 attackPercentage 3....,cup 1.0 dendroBonus 3....,head 1.0 critical 3....,attackPercentage 1.000 elementalMastery ...,5.487216,5.487216,8.787216,8.787216,8.395608


In [30]:
str_build_df = pd.read_json(proDir + '/config/expend_build.json')

NameError: name 'str_build_df' is not defined

In [15]:
for i,row in raw_build_data.iterrows():
    break

In [16]:
build = row

In [17]:
temp = build.fillna(value=0)
# 获得buildName
if temp[3] == 0:
    set_name = temp[2]
else:
    set_name = '{}、{}'.format(temp[2], temp[3])
if temp[1] == 0:
    build_name = '{}--{}--<{}{}{}>'.format(temp[0], set_name, temp[4], temp[5], temp[6])
else:
    build_name = '{}--{}--{}--<{}{}{}>'.format(temp[0], temp[1], set_name, temp[4], temp[5], temp[6])
expanded_build = {'buildName':build_name}

# 获得sets
sets = []
sets += set_simple[temp[2]]
if temp[3] != 0:
    sets += set_simple[temp[3]]
expanded_build['sets'] = sets

In [19]:
# 获得各部位主属性列表
flower_main_att = ['flower', 'lifeStatic']
feather_main_att = ['feather', 'attackStatic']
sand_main_att = ['sand'] + main_simple[build[4]]
cup_main_att = ['cup'] + main_simple[build[5]]
head_main_att = ['head'] + main_simple[build[6]]

In [20]:
# 生成有效属性列表
temp = build[7:14].dropna(inplace=False).index.tolist()
eff_attribute = []
for simple in temp:
    eff_attribute.append(eff_simple[simple])

In [22]:
# 生成build毕业难度列表
expanded_build['flowerDifficulty'] = rarity_of_build(flower_main_att, eff_attribute)
expanded_build['featherDifficulty'] = rarity_of_build(feather_main_att, eff_attribute)
expanded_build['sandDifficulty'] = rarity_of_build(sand_main_att, eff_attribute)
expanded_build['cupDifficulty'] = rarity_of_build(cup_main_att, eff_attribute)
expanded_build['headDifficulty'] = rarity_of_build(head_main_att, eff_attribute)

In [ ]:

# 生成各部位主属性评分权重 *5
expanded_build['flowerMainWeights'] = dict(zip(flower_main_att,[1,0]))
expanded_build['featherMainWeights'] = dict(zip(feather_main_att,[1,0]))
expanded_build['sandMainWeights'] = dict(zip(sand_main_att,[1,3.3]))
expanded_build['cupMainWeights'] = dict(zip(cup_main_att,[1, 3.3]))
if len(head_main_att) == 2:
    expanded_build['headMainWeights'] = dict(zip(head_main_att,[1, 3.3]))
else:
    expanded_build['headMainWeights'] = dict(zip(head_main_att,[1, 3.3, 3.3]))
# 各部位主属性评分权重补完 *3
for position in positions[2:]:
    temp = '{}MainWeights'.format(position)
    inborn = copy.deepcopy(score_weights.loc[position]).dropna()
    for eff in eff_attribute:  # 对于每一个有效属性
        if eff in expanded_build[temp]:  # drop掉存在于已有权重中的对应【固有】主属性
            inborn.drop(eff, inplace=True)
    for att in inborn.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
        if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
            inborn.drop(att, inplace=True)
    expanded_build[temp] = expanded_build[temp].update(inborn)
# 生成副属性评分权重
temp_sec = copy.deepcopy(score_weights.loc['sec']).dropna()
for att in temp_sec.index.tolist():
    if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
        temp_sec.drop(att, inplace=True)
expanded_build['secWeights'] = temp_sec
# 计算最佳得分
for position in positions:
    temp = '{}MainWeights'.format(position)
    eff_main = expanded_build[temp][1:2]
    sec = copy.deepcopy(score_weights.loc['sec']).dropna()
    for att in sec.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
        if att not in eff_attribute:  # 如果它不存在于有效属性中，则drop掉
            sec.drop(att, inplace=True)
    for att in sec.index.tolist():  # 对每一个固有主属性（已经去掉已有权重的）
        if att in eff_main:  # 如果它存在于主属性中，则drop掉
            sec.drop(att, inplace=True)
    if len(sec) > 4:
        full_sec = sec.sort_values(ascending=False)[0:4]
    else:
        full_sec = sec.sort_values(ascending=False)
    #best_att = pd.concat([eff_main, full_sec])
    best_score = eff_main[0] + 1.176 * full_sec.sum()
    expanded_build['best_{}'.format(position)] = best_score

In [3]:
import pandas as pd

good = pd.read_json('good.json')

ValueError: All arrays must be of the same length

In [11]:
with open('mona.json', 'r', encoding='utf8') as jd:
    art_data = json.load(jd)

artifacts = jsonpath(art_data, "$.flower[*]") + jsonpath(art_data, "$.feather[*]") + \
            jsonpath(art_data, "$.sand[*]") + jsonpath(art_data, "$.cup[*]") + jsonpath(art_data, "$.head[*]")
positions = ['flower', 'feather', 'sand', 'cup', 'head']

In [36]:
f['slotKey'].value_counts()

goblet     26
plume      25
sands      20
circlet    15
flower     12
Name: slotKey, dtype: int64